In [ ]:
import os
DEBUG = True
try:
    print("Original working directory: %s"%str(original_wd)) # type: ignore
    """
        You only get her0 if you---by mistake or otherwise---are re-running this cell, 
        in which case the working should not be changed again.
    """
except NameError:
    original_wd = os.getcwd()
    os.chdir('../')
print("Current working directory: %s"%str(os.getcwd()))

In [ ]:
from utils.data_handling import load_data, get_combined_cov_pos

In [ ]:
data = load_data()

from copy import copy
data_backup = copy(data)

In [ ]:
event_id = 0

data = data[data.event_id == event_id]
covs = get_combined_cov_pos(data)
data

In [ ]:
from utils.linalg import so3_to_su2, to_so3
from encoding import encode 
import pennylane.numpy as np
import pennylane as qml

In [ ]:
selected_ind = [0, 1]
wires = np.arange(5 * len(selected_ind))
dev = qml.device('default.qubit', wires=wires)

def layer(params):
    qml.templates.AngleEmbedding(params[:5], wires[:5], rotation='X')
    qml.templates.AngleEmbedding(params[5:10], wires[:5], rotation='Z')
    qml.templates.AngleEmbedding(params[10:15], wires[:5], rotation='X')
    # entangle position t0 with t1
    for ind in np.arange(3):
        q0 = wires[ind]
        q1 = wires[ind+5]
        qml.CNOT(wires=[q0, q1])
    # entangle cov(t0) with pos(t1)
    for ind in np.arange(3, 5):
        q0 = wires[ind]
        for jnd in np.arange(5, 8):
            q1 = wires[jnd]
            qml.CNOT(wires=[q0,q1])
    # entangle cov(t1) with pos(t1)
    for ind in np.arange(8, 10):
        q0 = wires[ind]
        for jnd in np.arange(5, 8):
            q1 = wires[jnd]
            qml.CNOT(wires=[q0,q1])
    
    qml.templates.AngleEmbedding(params[15:18], wires[5:8], rotation='X')
    qml.templates.AngleEmbedding(params[18:21], wires[5:8], rotation='Z')
    qml.templates.AngleEmbedding(params[21:24], wires[5:8], rotation='X')

@qml.qnode(dev)
def circ(covs, params):
    for ind, cov in enumerate(covs):
        encode(cov, wires=wires[5*ind:5*(ind+1)])
    layer(params)
    return [qml.expval(qml.PauliZ(wires=i)) for i in wires[5:8]]

def model(covs, params):
    out = circ(covs, params)
    alphas = 2*np.arccos(np.sqrt(.5 * (out[:3] + 1)))
    return alphas

circ(covs[:2], np.arange(24))
print(circ.draw())

In [ ]:
from collections import defaultdict

data = data_backup
covs = get_combined_cov_pos(data)

In [ ]:
events = defaultdict(list)
for (_,row), cov in zip(data.iterrows(), covs):
    events[row.event_id].append(cov)

In [ ]:
training_x = []
training_y = []
for event_id, cdms in events.items():
    if len(cdms) < 3:
        continue
    ind = 0
    while ind + 3 <= len(cdms):
        fst, snd, thd = cdms[ind:ind+3]
        training_x.append((fst, snd))
        eigs, _ = np.linalg.eigh(thd)
        eigs = np.log(eigs)/20
        training_y.append(eigs)
        ind += 3
len(training_x), len(training_y)

In [ ]:
np.random.seed(145315)
params = np.random.rand(30)

opt = qml.AdamOptimizer(0.3)

training_x = training_x[:100]
training_y = training_y[:100]

def cost(x, y, params):
    out = model(x, params)
    return np.linalg.norm(out - y)**2

def accuracy(train_x, train_y, params):
    mse = 0
    for x,y in zip(train_x, train_y):
        mse += np.sqrt(cost(x, y, params)) / np.linalg.norm(y)
    return mse / len(train_x)

n_epoch = 100
for epoch in np.arange(n_epoch):
    def obj_fn(params):
        loss = 0
        for x, y in zip(training_x, training_y):
            loss += cost(x, y, params)
        return loss
    params, current_loss = opt.step_and_cost(obj_fn, params)
    mse = accuracy(training_x, training_y, params)

    print(f"Epoch {epoch}: {mse:.1%}")